In [ ]:
print('Lets build the model :)')
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
#In this modeling approach we are taking 3 previous and trying to predict the 4th word in the sequance

Lets build the model :)


In [12]:
# reading the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [13]:
len(words)

32033

In [25]:
# building the mappings of characters to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
#building the dataset
block_size = 3 #context length how many characters do we take to predict the next one
X,Y = [],[]
for w in words[:5]:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)
        

In [62]:
#creating a lookup table for the character embeddings
C = torch.randn((27,2))
emb = C[X]

In [63]:
emb.shape

torch.Size([32, 3, 2])

In [71]:
# Constructiong the hidden layer
#The number of inputs to this layer is going to be 3 * 2  because we hae to dimensional embeddings and we have 3 of them
# and its up to us to decide how many neurons we want inside the layer here we are going with 100 of them
W1 = torch.randn(6,100)
b1 = torch.tensor(100)

In [ ]:
emb.view(32,6)
# here you can also use emb.reshape(32,6) the difference is emb.view() will not use extra space it ensure that the emb tensor and and the new tensor that we create will use the same data so no memory wastage


In [ ]:
torch.cat(torch.unbind(emb,1),1) == emb.view(32,6)
#here torch.unbind unbinds the tensor into 3 parts because we have passed 1 to it and the shape of current tensor is 32,3,2 and the indexing 1 corresponds to 3
# and then torch.cat() concatinates the 3 vector and we pass it 1 because we want to make it concatinate along axis 1 

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T